# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770780502491                   -0.53    8.0         
  2   -2.772128630957       -2.87       -1.30    1.0    127ms
  3   -2.772168970720       -4.39       -2.72    1.0    105ms
  4   -2.772170706299       -5.76       -3.48    2.0    139ms
  5   -2.772170719944       -7.87       -3.78    1.0    109ms
  6   -2.772170722720       -8.56       -4.56    1.0    110ms
  7   -2.772170722987       -9.57       -4.80    2.0    140ms
  8   -2.772170723014      -10.57       -5.89    1.0    114ms
  9   -2.772170723015      -12.23       -6.21    2.0    145ms
 10   -2.772170723015   +    -Inf       -6.20    1.0    122ms
 11   -2.772170723015      -13.39       -6.39    1.0    122ms
 12   -2.772170723015   +  -12.78       -5.94    2.0    147ms
 13   -2.772170723015      -12.91       -6.20    1.0    123ms
 14   -2.772170723015      -13.34       -7.25    1.0    126ms
 15   -2.

1.7735581157003615

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770786330760                   -0.52    8.0         
  2   -2.772055701052       -2.90       -1.32    1.0    104ms
  3   -2.772082914057       -4.57       -2.51    1.0    135ms
  4   -2.772083346739       -6.36       -3.31    1.0    106ms
  5   -2.772083415349       -7.16       -3.82    2.0    123ms
  6   -2.772083417727       -8.62       -4.96    1.0    132ms
  7   -2.772083417807      -10.10       -5.24    2.0    125ms
  8   -2.772083417810      -11.47       -6.35    1.0    116ms
  9   -2.772083417811      -12.86       -6.74    2.0    135ms
 10   -2.772083417811      -14.57       -7.71    1.0    132ms
 11   -2.772083417811      -14.31       -8.17    2.0    132ms

Polarizability via ForwardDiff:       1.772534968445816
Polarizability via finite difference: 1.7735581157003615
